In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)


In [ ]:
print(f"Read {train['review'].size} labeled train reviews, {test['review'].size} labeled test reviews, and {unlabeled_train['review'].size} unlabeled reviews\n")

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [ ]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
  review_text = BeautifulSoup(review).get_text()  #HTML 제거
  review_text = re.sub("[^a-zA-Z]", " ", review_text) #불용어 제거
  words = review_text.lower().split() #소문자로 전환

  if remove_stopwords:  #stop words 제거(defalut는 false)
    stops = stopwords.words("english")
    words = [w for w in words if not w in stops]

  return(words)


In [ ]:
import nltk.data
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> ㅇ
Command 'ㅇ' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger is already up-to-date!
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_ru is already up-to-
       |       date!
       | Downloading package basque_grammars to /root/nltk_data...
       |   Package basque_grammars is already up-to-date!
       | Downloading package bcp47 to /root/nltk_data...
       |   Package bcp47 is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Packag


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords=False):
  raw_sentences = tokenizer.tokenize(review.strip())
  sentences = []
  for raw_sentence in raw_sentences :
    if len(raw_sentence) > 0:
      sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))

  return sentences

In [ ]:
sentences = []

print("Parsing sentences from training set")
for review in train["review"]:
  sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
  sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


<ipython-input-25-d7f176c101bf>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()  #HTML 제거
<ipython-input-25-d7f176c101bf>:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()  #HTML 제거


Parsing sentences from unlabeled set


In [ ]:
print(len(sentences))

796172


In [ ]:
print(sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [36]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, vector_size = num_features,
                          min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)



Training model...


<ipython-input-36-67307798d285>:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [38]:
model_name = "300features_40minwords_10context"
model.save(model_name)

#from gensim.models import KeyedVectors
#model.wv.save_word2vec_format('eng_w2v') # 모델 저장
#loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [42]:
from gensim.models import Word2Vec

model_path = "/content/300features_40minwords_10context"
model = Word2Vec.load(model_path)

In [44]:
import numpy as np

In [60]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index_to_key)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model.wv.get_vector(word))
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print(f"Review {counter} of {len(reviews)}")
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs



In [61]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )


<ipython-input-25-d7f176c101bf>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()  #HTML 제거


Review 0.0 of 25000
Review 1000.0 of 25000
Review 2000.0 of 25000
Review 3000.0 of 25000
Review 4000.0 of 25000
Review 5000.0 of 25000
Review 6000.0 of 25000
Review 7000.0 of 25000
Review 8000.0 of 25000
Review 9000.0 of 25000
Review 10000.0 of 25000
Review 11000.0 of 25000
Review 12000.0 of 25000
Review 13000.0 of 25000
Review 14000.0 of 25000
Review 15000.0 of 25000
Review 16000.0 of 25000
Review 17000.0 of 25000
Review 18000.0 of 25000
Review 19000.0 of 25000
Review 20000.0 of 25000
Review 21000.0 of 25000
Review 22000.0 of 25000
Review 23000.0 of 25000
Review 24000.0 of 25000
Creating average feature vecs for test reviews
Review 0.0 of 25000
Review 1000.0 of 25000
Review 2000.0 of 25000
Review 3000.0 of 25000
Review 4000.0 of 25000
Review 5000.0 of 25000
Review 6000.0 of 25000
Review 7000.0 of 25000
Review 8000.0 of 25000
Review 9000.0 of 25000
Review 10000.0 of 25000
Review 11000.0 of 25000
Review 12000.0 of 25000
Review 13000.0 of 25000
Review 14000.0 of 25000
Review 15000.0 of 2

In [62]:

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results
result = forest.predict( testDataVecs )

# Write the test results
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )


Fitting a random forest to labeled training data...
